In [ ]:
!pip install databricks-sql
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pystache: filename=pystache-0.6.0-py3-none-any.whl size=83618 sha256=af3ccf97fd45eb9a1c088e9eb2946a665e8f46fe19cb8dffcf30774c015fb8f6
  Stored in directory: /root/.cache/pip/wheels/4a/98/2c/84a50139380b5a5d8c0b4dfb92f2d860953c9e8e755507a402
  Created wheel for thrift: filename=thrift-0.16.0-cp310-cp310-linux_x86_64.whl size=373871 sha256=ef06602154fe9dc4cb37ba528df199698c6e0d6a642f4aa4d54869f43287e9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from databricks import sql
import pandas as pd
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA


def get_flight_data(source,destination,airline):
    connection = sql.connect(server_hostname = "",
                 http_path       = "",
                 access_token    = "")
    query = f"""SELECT id, from, date_from, to, price,
              airline FROM flight_recommendation.default.flight
              where '{source}' = from
              and '{destination}'= to and airline = '{airline}'
            """
    result = pd.read_sql(query, connection)
    result['price']=result['price'].astype(float)
    min_prices = result.groupby('date_from')['price'].min().reset_index()
    min_prices['price_diff'] = min_prices['price'] - 2 * min_prices['price'].shift(1) + min_prices['price'].shift(2)
    min_prices = min_prices.dropna()
    return min_prices


def arima_model(data,forecast):

    model = ARIMA(data['price'], order=(1, 1, 1))
    results = model.fit()
    if forecast == 0.0:
      forecast_steps = 10
      predictions = results.get_prediction(start=len(data), end=len(data) + forecast_steps - 1)
      return pd.DataFrame(predictions.predicted_mean)
    else:
      forecast_steps = int(forecast)
      predictions = results.get_prediction(start=len(data), end=len(data) + forecast_steps - 1)
      return pd.DataFrame(predictions.predicted_mean)


def plot_predictions(data, predictions):
    data['data'] = 'historic'
    predictions['data'] = 'forecast'

    last_date = data['date_from'].max()
    last_date = pd.to_datetime(last_date)

    historic_df = data[['date_from', 'price', 'data']]
    forecast_df = pd.DataFrame({
        'date_from': pd.date_range(start=last_date + pd.to_timedelta(1, unit='D'), periods=len(predictions)),
        'predict': predictions.predicted_mean,
        'data': 'forecast'
    })

    df = pd.concat([historic_df, forecast_df], ignore_index=True)

    if df['predict'].empty:
      st.warning("No Data Available")
    else:
      today_date = datetime.today().strftime('%Y-%m-%d')
      df['date_from'] = pd.to_datetime(df['date_from'])
      data = df[df['date_from'] >= today_date]

      fig = px.line(data, x="date_from", y="price")
      fig.add_scatter(x=data["date_from"], y=data["price"], mode="markers",name='Historic')
      fig.add_scatter(x=data["date_from"], y=data["predict"], mode="lines+markers",name='Forecast')
      st.plotly_chart(fig)

st.title("Flight Price Prediction")

source = ["New York, NY (all airports)","Los Angeles, CA (LAX)",
"Chicago, IL (all airports)","Houston, TX (all airports)"]
destination = [
    "Phoenix, AZ (all airports)", "San Diego, CA (SAN)", "Chicago, IL (all airports)",
    "Houston, TX (all airports)", "Los Angeles, CA (LAX)", "New York, NY (all airports)",
    "Columbus, OH (CMH)", "Philadelphia, PA (PHL)", "Charlotte, NC (CLT)",
    "Dallas, TX (all airports)", "San Jose, CA (SJC)", "Jacksonville, FL (JAX)",
    "Indianapolis, IN (IND)", "Austin, TX (AUS)", "San Francisco, CA (SFO)",
    "San Antonio, TX (SAT)", "Fort Worth (DFW)", "Seattle, WA (SEA)",
    "Denver, CO (DEN)"
]

source = st.sidebar.selectbox("Enter the Source City",source)
destination = st.sidebar.selectbox("Enter the Destination City",destination)

airline = ['Spirit','American','United','Delta','JetBlue','Multiple Airlines','Alaska',
           'Frontier','Southwest','Other Airline','Allegiant','Sun Country Airlines',
           'Southern Airways Express']

airline = st.sidebar.selectbox("Select Airline", airline)
data = get_flight_data(source,destination,airline)

forecast = st.number_input("Enter Days To Be Forecasted For :",value=10)

predictions = arima_model(data,forecast)

plot_predictions(data, predictions)




Overwriting app.py


In [ ]:
# Create a package.json file with default values
!echo '{"name": "colab", "version": "1.0.0", "main": "index.js", "scripts": {"start": "echo \"Error: no start script specified\" && exit 1"}, "keywords": [], "author": "", "license": "ISC"}' > /content/package.json

# Install localtunnel
!npm install localtunnel

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN colab@1.0.0 No description
npm WARN colab@1.0.0 No repository field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.14s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.123.0.58
npx: installed 22 in 4.425s
your url is: https://evil-ads-fold.loca.lt
